In [3]:
from keras.preprocessing.image import load_img 
from keras.preprocessing.image import img_to_array 
from keras.applications.vgg16 import preprocess_input 

# models 
from keras.applications.vgg16 import VGG16 
from keras.models import Model

from sklearn.decomposition import PCA

# for everything else
import os
import numpy as np
import matplotlib.pyplot as plt
from random import randint
import pandas as pd
import pickle


In [4]:
cd

/Users/valencialie


In [ ]:
cd Desktop/CZ1016_DS2/kdrama/Images/Male

In [5]:
path = r"/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Male"
# change the working directory to the path where the images are located
os.chdir(path)

In [6]:
actors = [] #create an empty list to store all the image files

In [7]:
# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            actors.append(file.name)

In [8]:
len(actors)

576

In [9]:
cd

/Users/valencialie


In [10]:
cd Downloads/Images/Male

/Users/valencialie/Downloads/Images/Male


In [159]:
model = VGG16()
model = Model(inputs = model.inputs, outputs = model.layers[-2].output)

def extract_features(file, model):
    # load the image as a 224x224 array
    img = load_img(file, target_size=(224,224))
    # convert from 'PIL.Image.Image' to numpy array
    img = np.array(img) 
    # reshape the data for the model reshape(num_of_samples, dim 1, dim 2, channels 1 cause grayscale)
    reshaped_img = img.reshape(1,224,224,3) 
    # prepare image for model
    imgx = preprocess_input(reshaped_img)
    # get the feature vector
    features = model.predict(imgx, use_multiprocessing=True)
    return features

In [12]:
data = {}

# lop through each image in the dataset
for actor in actors:
    # try to extract the features and update the dictionary
    try:
        feat = extract_features(actor,model)
        data[actor] = feat
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        pass
          
# get a list of the filenames
filenames = np.array(list(data.keys()))

# get a list of just the features
feat = np.array(list(data.values()))
feat.shape

feat = feat.reshape(-1,4096)
feat.shape

(576, 4096)

In [13]:
pca = PCA(n_components=100, random_state=22)
pca.fit(feat)
x = pca.transform(feat)



In [14]:
len(x)

576

In [15]:
array_list = list(x)


In [16]:
array_label = list(data.keys())
d = { label: value for label, value in zip(array_label, array_list) }

In [17]:
d

{'Park_Hae_Soo.png': array([-4.117684  , -9.592239  , -7.871775  ,  1.1058365 ,  5.1641855 ,
         4.068646  ,  8.857245  ,  6.075376  , -8.628299  ,  7.500619  ,
         8.490868  , -0.746574  ,  7.535375  ,  1.424     ,  3.1271615 ,
        -4.435749  ,  2.1409383 ,  2.357074  , -0.08353359,  1.8660791 ,
        -3.6820078 ,  6.48838   ,  1.3201274 , -0.9451957 ,  0.1001891 ,
         0.40318888, -2.7479167 ,  7.0380497 ,  5.1558437 ,  1.0104094 ,
         2.597317  , -5.121636  , -2.4225447 ,  0.9487666 ,  2.7827344 ,
        -2.3580527 , -1.5409752 ,  3.05039   , -1.0844706 ,  2.648658  ,
         4.1342964 ,  0.8826878 ,  0.81001186,  1.5107691 , -1.3215538 ,
        -0.93360144, -0.57638776, -5.2344856 ,  0.6601448 ,  1.732518  ,
        -1.2057248 , -1.0907246 ,  2.5953574 , -0.345987  ,  2.2580347 ,
         1.0881535 , -1.9658955 , -1.8751581 ,  1.5552125 , -1.2997589 ,
        -0.3780705 , -0.27423128, -2.674555  ,  0.66701365, -1.3713564 ,
         0.77248865,  1.7554336

In [18]:
df = pd.DataFrame(d.items(), columns=['Actor', 'Array'])

In [102]:
dist_list = []
dist_list2 = []
index = []
zipped_lists = []
sorted_zipped_lists = []
top5 = []
overall = []

In [103]:
for j in range(0,576):
    for i in range(0,576):
        if i == j:
            continue
        else:
            dist = np.linalg.norm(x[j]-x[i])
            dist_list.append(dist)
            dist_list2.append(dist)
            index.append(i)
            dist_list.sort()
            top5 = dist_list[:5]
            zipped_lists = zip(dist_list2, index)
            sorted_zipped_lists = sorted(zipped_lists)
            sorted_list1 = [element for _, element in sorted_zipped_lists]
            top5.extend(sorted_list1[:5])
    overall.append(top5)
    dist_list = []
    dist_list2 = []
    index = []
    zipped_lists = []
    sorted_list1 =[]
    top5 = []

In [140]:
# overall 

In [105]:
df1 = pd.DataFrame(overall,columns=['Least_Array_Diff', '2ndLeast_Array_Diff','3rdLeast_Array_Diff', '4thLeast_Array_Diff', '5thLeast_Array_Diff', '1stMatch', '2ndMatch', '3rdMatch', '4thMatch', '5thMatch'])

In [107]:
df2 = pd.concat([df, df1], axis = 1)

In [108]:
df2

,Actor,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch
0,Park_Hae_Soo.png,"[-4.117684, -9.592239, -7.871775, 1.1058365, 5...",30.715021,30.962044,33.221836,35.256233,35.511070,73,399,175,488,406
1,Lee_Jin_Wook.png,"[65.89964, 15.4892235, -10.712205, -14.894704,...",39.949093,44.084579,47.698715,48.780590,49.159027,519,390,365,141,37
2,Kang_Min_Hyuk.png,"[-12.266893, -7.2090816, 1.4790905, 9.8934355,...",26.389797,30.042631,30.068354,31.712767,31.934317,276,112,314,216,351
3,Yoon_Park.png,"[-18.531435, 2.268928, -6.4732237, -1.7551794,...",30.235329,32.937115,33.250229,35.058430,35.874107,379,242,383,95,152
4,Joo_Won.png,"[-2.5595207, 15.16615, -0.14534593, -0.7223856...",22.880953,25.406965,27.186726,27.307686,27.335983,199,243,567,535,449
...,...,...,...,...,...,...,...,...,...,...,...,...
571,Lee_Dong_Gun.png,"[-4.8451734, -6.19492, -6.385351, 7.14666, -7....",29.526987,30.920572,32.103840,32.242767,32.314220,215,399,272,359,60
572,Han_Seok_Kyu.png,"[37.471012, -9.558895, -1.0444481, -1.9539661,...",29.246450,31.108734,31.635012,31.926279,32.351116,55,220,358,325,137
573,Choi_Byung_Chan.png,"[-13.009411, -10.179055, 26.422827, -3.3712656...",45.112774,48.375237,50.387230,51.085152,51.381248,64,570,528,9,118
574,Lee_Won_Jong.png,"[21.324755, -0.48560676, -14.931668, -23.75189...",36.048111,41.989658,42.230412,48.440102,49.800938,446,282,288,187,191


In [128]:
bestmatch = []

for i in range(0,576):
    bestmatch.append(df2.iloc[df2['1stMatch'][i], 0])
    

In [129]:
secondmatch = []

for i in range(0,576):
    secondmatch.append(df2.iloc[df2['2ndMatch'][i], 0])

In [130]:
thirdmatch = []

for i in range(0,576):
    thirdmatch.append(df2.iloc[df2['3rdMatch'][i], 0])

In [131]:
fourthmatch = []

for i in range(0,576):
    fourthmatch.append(df2.iloc[df2['4thMatch'][i], 0])

In [132]:
fifthmatch = []

for i in range(0,576):
    fifthmatch.append(df2.iloc[df2['5thMatch'][i], 0])

In [136]:
df3 = pd.DataFrame(bestmatch,columns=['Best_Match_Actor'])
df4 = pd.DataFrame(secondmatch,columns=['Second_Match_Actor'])
df5 = pd.DataFrame(thirdmatch,columns=['Third_Match_Actor'])
df6 = pd.DataFrame(fourthmatch,columns=['Fourth_Match_Actor'])
df7 = pd.DataFrame(fifthmatch,columns=['Fifth_Match_Actor'])

In [137]:
df8 = pd.concat([df2, df3, df4, df5, df6, df7], axis = 1)

In [138]:
df8

,Actor,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch,Best_Match_Actor,Second_Match_Actor,Third_Match_Actor,Fourth_Match_Actor,Fifth_Match_Actor
0,Park_Hae_Soo.png,"[-4.117684, -9.592239, -7.871775, 1.1058365, 5...",30.715021,30.962044,33.221836,35.256233,35.511070,73,399,175,488,406,Yeon_Jung_Hoon.png,Joo_Jin_Mo.png,Ahn_Nae_Sang.png,Ji_Hyun_Woo.png,Shin_Hyun_Soo.png
1,Lee_Jin_Wook.png,"[65.89964, 15.4892235, -10.712205, -14.894704,...",39.949093,44.084579,47.698715,48.780590,49.159027,519,390,365,141,37,Kim_Sang_Joong.png,Lee_Bum_Soo.png,Kim_Myung_Min.png,Hong_Kyung_Min.png,Lee_Ji_Hoon.png
2,Kang_Min_Hyuk.png,"[-12.266893, -7.2090816, 1.4790905, 9.8934355,...",26.389797,30.042631,30.068354,31.712767,31.934317,276,112,314,216,351,Lee_Jung_Jin.png,Lee_Jun_Ho.png,Yoo_Yeon_Seok.png,Kim_Jung_Hyun.png,Lee_Seung_Min.png
3,Yoon_Park.png,"[-18.531435, 2.268928, -6.4732237, -1.7551794,...",30.235329,32.937115,33.250229,35.058430,35.874107,379,242,383,95,152,Ohn_Joo_Wan.png,Hoya.png,Choi_Byung_Mo.png,Hong_Jong_Hyun.png,Lee_Gi_Kwang.png
4,Joo_Won.png,"[-2.5595207, 15.16615, -0.14534593, -0.7223856...",22.880953,25.406965,27.186726,27.307686,27.335983,199,243,567,535,449,Kim_Seon_Ho.png,Lee_Dong_Wook.png,Kwon_Yool.png,Yoon_Shi_Yoon.png,Noh_Jong_Hyun.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
571,Lee_Dong_Gun.png,"[-4.8451734, -6.19492, -6.385351, 7.14666, -7....",29.526987,30.920572,32.103840,32.242767,32.314220,215,399,272,359,60,Lee_Jae_Won.png,Joo_Jin_Mo.png,Park_Byung_Eun.png,Ahn_Seung_Gyun.png,Lee_Kang_Min.png
572,Han_Seok_Kyu.png,"[37.471012, -9.558895, -1.0444481, -1.9539661,...",29.246450,31.108734,31.635012,31.926279,32.351116,55,220,358,325,137,Kim_Gab_Soo.png,Kim_Sun_Hyuk.png,Kim_Yeong_Cheol.png,Kim_Min_Joon.png,Yoo_Ji_Tae.png
573,Choi_Byung_Chan.png,"[-13.009411, -10.179055, 26.422827, -3.3712656...",45.112774,48.375237,50.387230,51.085152,51.381248,64,570,528,9,118,Park_Geon_Rak.png,Park_Ji_Bin.png,Kim_Young_Kwang.png,Shin_Dong_Woo.png,Seo_Hyun_Chul.png
574,Lee_Won_Jong.png,"[21.324755, -0.48560676, -14.931668, -23.75189...",36.048111,41.989658,42.230412,48.440102,49.800938,446,282,288,187,191,Oh_Chang_Seok.png,Lee_Jae_Ryong.png,Yoon_Jung_Il.png,Kim_Young_Min.png,Jung_Kyung_Ho.png


In [139]:
path = r"/Users/valencialie/Desktop/CZ1016_DS2/kdrama/Images/Female"
# change the working directory to the path where the images are located
os.chdir(path)

In [142]:
actresses = []

In [143]:
# creates a ScandirIterator aliased as files
with os.scandir(path) as files:
  # loops through each file in the directory
    for file in files:
        if file.name.endswith('.png'):
          # adds only the image files to the flowers list
            actresses.append(file.name)

In [174]:
len(actresses)

474

In [175]:
cd

/Users/valencialie


In [176]:
cd Desktop/CZ1016_DS2/kdrama/Images/Female

/Users/valencialie/Downloads/Images/Female


In [177]:
dataF = {}

In [178]:
# loop through each image in the dataset
for actress in actresses:
    # try to extract the features and update the dictionary
    try:
        featF = extract_features(actress,model)
        dataF[actress] = featF
    # if something fails, save the extracted features as a pickle file (optional)
    except:
        pass
          
# get a list of the filenames
filenamesF = np.array(list(dataF.keys()))

# get a list of just the features
featF = np.array(list(dataF.values()))
featF.shape

featF = featF.reshape(-1,4096)
featF.shape

(474, 4096)

In [179]:
pca = PCA(n_components=100, random_state=22)
pca.fit(featF)
xF = pca.transform(featF)

In [180]:
len(xF)

474

In [187]:
array_listF = list(xF)

In [189]:
array_labelF = list(dataF.keys())
F = { label: value for label, value in zip(array_labelF, array_listF) }

In [190]:
F

{'Jeon_Hye_Jin.png': array([ 1.00570374e+01, -5.93734932e+00,  3.45059931e-01, -1.20929985e+01,
        -1.19697742e+01,  2.22610736e+00, -1.41368699e+00, -5.02225339e-01,
        -3.33084369e+00, -6.13454938e-01, -1.38263476e+00, -6.94341242e-01,
         4.81181192e+00, -5.56591940e+00,  8.38479877e-01,  3.01680475e-01,
         2.02158689e+00,  1.40618825e+00, -1.01724112e+00,  1.96578312e+00,
         1.54479489e-01, -8.47938061e-01,  1.35611081e+00, -6.27312660e+00,
         3.30300808e+00, -3.22625470e+00, -1.15838134e+00, -8.03789437e-01,
         1.05162132e+00,  4.16654205e+00, -8.62359166e-01,  8.62668514e-01,
        -1.98969674e+00, -3.89172792e+00, -7.69078255e-01, -1.83258545e+00,
         2.23917294e+00, -6.28090873e-02,  4.13293982e+00,  3.07687342e-01,
        -1.01895738e+00, -1.83555400e+00,  3.48235428e-01,  2.16518998e+00,
        -3.51444030e+00, -1.48363495e+00, -7.48185575e-01, -9.03674424e-01,
         1.81494129e+00,  4.61744756e-01,  8.20232928e-01, -3.761733

In [193]:
dF = pd.DataFrame(F.items(), columns=['Actress', 'Array'])

In [194]:
dF

,Actress,Array
0,Jeon_Hye_Jin.png,"[10.057037, -5.9373493, 0.34505993, -12.092998..."
1,Kim_Hyun_Soo.png,"[2.198409, -10.022728, 0.036321774, 4.9317007,..."
2,Im_Soo_Jung.png,"[-2.597886, -12.319992, 9.091196, -1.5792747, ..."
3,Bae_Hae_Seon.png,"[24.630611, 0.7530353, -2.7690136, -22.539837,..."
4,Choi_Kang_Hee.png,"[10.8650255, -0.5788472, 13.763874, 1.0320615,..."
...,...,...
469,Kwon_Na_Ra.png,"[8.329456, -4.3880363, 4.933248, 4.911453, 4.2..."
470,Lee_Moon_Jung.png,"[-10.568473, 20.176798, 5.152825, 23.702711, -..."
471,Gong_Seung_Yeon.png,"[-5.9241323, 2.2409234, 4.177446, -11.999601, ..."
472,Lee_Soo_Kyung.png,"[1.5862242, -8.364018, -17.490177, -9.999423, ..."


In [195]:
dist_list = []
dist_list2 = []
index = []
zipped_lists = []
sorted_zipped_lists = []
top5 = []
overallF = []

In [196]:
for j in range(0,474):
    for i in range(0,474):
        if i == j:
            continue
        else:
            dist = np.linalg.norm(xF[j]-xF[i])
            dist_list.append(dist)
            dist_list2.append(dist)
            index.append(i)
            dist_list.sort()
            top5 = dist_list[:5]
            zipped_lists = zip(dist_list2, index)
            sorted_zipped_lists = sorted(zipped_lists)
            sorted_list1 = [element for _, element in sorted_zipped_lists]
            top5.extend(sorted_list1[:5])
    overallF.append(top5)
    dist_list = []
    dist_list2 = []
    index = []
    zipped_lists = []
    sorted_list1 =[]
    top5 = []

In [198]:
dF1 = pd.DataFrame(overallF,columns=['Least_Array_Diff', '2ndLeast_Array_Diff','3rdLeast_Array_Diff', '4thLeast_Array_Diff', '5thLeast_Array_Diff', '1stMatch', '2ndMatch', '3rdMatch', '4thMatch', '5thMatch'])

In [200]:
dF2 = pd.concat([dF, dF1], axis = 1)

In [201]:
dF2

,Actress,Array,Least_Array_Diff,2ndLeast_Array_Diff,3rdLeast_Array_Diff,4thLeast_Array_Diff,5thLeast_Array_Diff,1stMatch,2ndMatch,3rdMatch,4thMatch,5thMatch
0,Jeon_Hye_Jin.png,"[10.057037, -5.9373493, 0.34505993, -12.092998...",27.373398,27.758600,28.542065,28.774691,28.893175,349,47,413,376,275
1,Kim_Hyun_Soo.png,"[2.198409, -10.022728, 0.036321774, 4.9317007,...",17.859856,19.096994,20.494192,21.847498,23.016798,47,63,413,100,249
2,Im_Soo_Jung.png,"[-2.597886, -12.319992, 9.091196, -1.5792747, ...",27.188494,29.429840,29.550230,29.815596,30.265022,115,23,245,230,29
3,Bae_Hae_Seon.png,"[24.630611, 0.7530353, -2.7690136, -22.539837,...",28.366072,34.246185,37.451447,37.535973,37.550652,39,28,0,102,54
4,Choi_Kang_Hee.png,"[10.8650255, -0.5788472, 13.763874, 1.0320615,...",29.160507,31.255350,33.890415,34.438023,34.561924,235,352,88,167,140
...,...,...,...,...,...,...,...,...,...,...,...,...
469,Kwon_Na_Ra.png,"[8.329456, -4.3880363, 4.933248, 4.911453, 4.2...",25.251627,25.878393,25.987658,26.352592,27.485931,1,32,29,47,249
470,Lee_Moon_Jung.png,"[-10.568473, 20.176798, 5.152825, 23.702711, -...",25.378578,30.228170,30.628948,32.221382,32.580814,121,424,296,314,408
471,Gong_Seung_Yeon.png,"[-5.9241323, 2.2409234, 4.177446, -11.999601, ...",17.194269,17.260630,17.750561,19.639339,20.572678,256,230,306,23,415
472,Lee_Soo_Kyung.png,"[1.5862242, -8.364018, -17.490177, -9.999423, ...",30.634533,31.757921,32.362850,33.150856,33.356060,106,336,311,286,456


In [202]:
bestmatchF = []

for i in range(0,474):
    bestmatchF.append(dF2.iloc[dF2['1stMatch'][i], 0])

In [204]:
secondmatchF = []

for i in range(0,474):
    secondmatchF.append(dF2.iloc[dF2['2ndMatch'][i], 0])

In [205]:
thirdmatchF = []

for i in range(0,474):
    thirdmatchF.append(dF2.iloc[dF2['3rdMatch'][i], 0])
    
fourthmatchF = []

for i in range(0,474):
    fourthmatchF.append(dF2.iloc[dF2['4thMatch'][i], 0])
    
fifthmatchF = []

for i in range(0,474):
    fifthmatchF.append(dF2.iloc[dF2['5thMatch'][i], 0])

In [206]:
dF3 = pd.DataFrame(bestmatchF,columns=['Best_Match_Actress'])
dF4 = pd.DataFrame(secondmatchF,columns=['Second_Match_Actress'])
dF5 = pd.DataFrame(thirdmatchF,columns=['Third_Match_Actress'])
dF6 = pd.DataFrame(fourthmatchF,columns=['Fourth_Match_Actress'])
dF7 = pd.DataFrame(fifthmatchF,columns=['Fifth_Match_Actress'])

In [208]:
dF8 = pd.concat([dF2, dF3, dF4, dF5, dF6, dF7], axis = 1)

In [211]:
#dF8.to_csv("actress.csv")

In [213]:
#df8.to_csv("actor.csv")